In [9]:
import pandas as pd
import glob
import os
import numpy as np
import pickle as pkl
import time
from fastparquet import write
import fastparquet
import dask.dataframe as dd

In [2]:
all_pkl_files=[]
pkl_file_sizes=[]
path='/data/dharp/compounds/datasets/googleV3/'
for filename in glob.glob(path+'*pkl'):
    pkl_file_sizes.append(os.path.getsize(filename))
    all_pkl_files.append(filename)
#df = pd.concat(dfs)

In [3]:
pkl_df=pd.DataFrame(all_pkl_files,pkl_file_sizes)
pkl_df.reset_index(inplace=True)
pkl_df.columns=['fsize','fname']
pkl_df['fsize_perc']=pkl_df.fsize/pkl_df.fsize.sum()*100
pkl_df.sort_values(by=['fsize'],ascending=False,inplace=True,ignore_index=True)
pkl_df.fsize/=1024*1024
pkl_df

,fsize,fname,fsize_perc
0,4600.627284,/data/dharp/compounds/datasets/googleV3/805.pkl,1.247389
1,4525.094378,/data/dharp/compounds/datasets/googleV3/800.pkl,1.226910
2,4455.533458,/data/dharp/compounds/datasets/googleV3/816.pkl,1.208049
3,4365.063416,/data/dharp/compounds/datasets/googleV3/802.pkl,1.183520
4,4363.879601,/data/dharp/compounds/datasets/googleV3/819.pkl,1.183199
...,...,...,...
822,0.337672,/data/dharp/compounds/datasets/googleV3/2190.pkl,0.000092
823,0.279921,/data/dharp/compounds/datasets/googleV3/2408.pkl,0.000076
824,0.177505,/data/dharp/compounds/datasets/googleV3/2374.pkl,0.000048
825,0.166813,/data/dharp/compounds/datasets/googleV3/2151.pkl,0.000045


In [4]:
maxvalue = 30_000

lastvalue = 0
newcum = []
labels=[]
cur_label=1
for row in pkl_df.itertuples():
    thisvalue =  row.fsize + lastvalue
    if thisvalue > maxvalue:
        thisvalue = 0
        cur_label+=1
    newcum.append( thisvalue )
    labels.append(cur_label)
    lastvalue = thisvalue
pkl_df['fcat']=labels
pkl_df

,fsize,fname,fsize_perc,fcat
0,4600.627284,/data/dharp/compounds/datasets/googleV3/805.pkl,1.247389,1
1,4525.094378,/data/dharp/compounds/datasets/googleV3/800.pkl,1.226910,1
2,4455.533458,/data/dharp/compounds/datasets/googleV3/816.pkl,1.208049,1
3,4365.063416,/data/dharp/compounds/datasets/googleV3/802.pkl,1.183520,1
4,4363.879601,/data/dharp/compounds/datasets/googleV3/819.pkl,1.183199,1
...,...,...,...,...
822,0.337672,/data/dharp/compounds/datasets/googleV3/2190.pkl,0.000092,12
823,0.279921,/data/dharp/compounds/datasets/googleV3/2408.pkl,0.000076,12
824,0.177505,/data/dharp/compounds/datasets/googleV3/2374.pkl,0.000048,12
825,0.166813,/data/dharp/compounds/datasets/googleV3/2151.pkl,0.000045,12


results[0][['modifier','head']].value_counts().sample(50)

def write_to_hdf(data_bin):
    if data_bin.shape[0]!=0:
        df_list=[]
        print(data_bin.shape[0])
        for row in data_bin.itertuples():
            #print(row.fname)
            tmp_pkl=pd.read_pickle(row.fname)
            tmp_pkl.reset_index(inplace=True)
            df_list.append(tmp_pkl)
        tmp_df=pd.concat(df_list,ignore_index=True,sort=False)
        print('Done concatenating')
        tmp_df.groupby(['fivegram_pos','year'])['count'].sum().to_frame().reset_index(inplace=True)
        print('Done grouping')
        tmp_df.to_hdf(path+'/entire_df_'+str(row.fcat)+'.h5',format='table', key='df',complib='zlib', complevel=5)

In [5]:
def write_to_parquet(data_bin):
    print(data_bin.iloc[0].fcat)
    cur_time=time.time()
    df_list=[]
    print(data_bin.shape[0])
    for row in data_bin.itertuples():
        #print(row.fname)
        cur_df=pd.read_pickle(row.fname)
        cur_df.reset_index(inplace=True)
        df_list.append(cur_df)
    concat_df=pd.concat(df_list,ignore_index=True,sort=False)
    print(concat_df.shape[0])
    total_df_shape=concat_df.shape[0]
    print('Done concatenating')
    
    ddf = dd.from_pandas(concat_df, npartitions=30)
    
    ddf=ddf.groupby(['lemma_pos','pos_sent','year','comp_class'])['count'].sum()
    print('Done grouping')
    ddf=ddf.to_frame().reset_index().compute()
    
    print(ddf.shape[0])
    after_shape=ddf.shape[0]
    
    ddf.to_parquet(
    path=f'{save_path}/df_{row.fcat}.parq', 
    engine='fastparquet',
    compression='snappy',
    row_group_offsets=10_000_000)
    print(f"Finished df {row.fcat} ; Before : {total_df_shape}, After : {after_shape} Change in percentage : {(total_df_shape-after_shape)/total_df_shape*100:0.2f}%")
    print(f'Time taken {time.time()-cur_time} secs')

In [6]:
save_path='/data/dharp/compounds/datasets/entire_df'


In [7]:
pkl_df.groupby('fcat').apply(write_to_parquet)

1
6
954357570
Done concatenating
1
6
954357570
Done concatenating


NameError: name 'dd' is not defined

In [23]:
pkl_df.groupby('fcat')